In [1]:
import os
import time
import pandas as pd
import json
import requests
from geopy.geocoders import Nominatim
import geopandas as gpd

In [2]:
pd.set_option('display.max_columns',None)

## COAL PLANTS

In [3]:
url="https://greeninfo-network.github.io/coal-tracker-client/data/trackers.json"

In [4]:
response = requests.get(url)
if response.status_code == 200:
    json_data = json.loads(response.text)
json_data

[{'id': 2,
  'country': 'Argentina',
  'url': 'https://www.gem.wiki/R%C3%ADo_Turbio_power_station',
  'plant': 'Río Turbio power station',
  'unit': 'Unit 1',
  'plant_local': 'Central Térmica Río Turbio',
  'owner': 'Yacimientos Carboníferos de Río Turbio SA',
  'parent': 'Yacimientos Carboníferos de Río Turbio SA',
  'capacity': '120',
  'status': 'operating',
  'year': '2022',
  'subnational': 'Santa Cruz',
  'region': 'Americas',
  'lat': -51.54601,
  'lng': -72.23126},
 {'id': 3,
  'country': 'Argentina',
  'url': 'https://www.gem.wiki/R%C3%ADo_Turbio_power_station',
  'plant': 'Río Turbio power station',
  'unit': 'Unit 2',
  'plant_local': 'Central Térmica Río Turbio',
  'owner': 'Yacimientos Carboníferos de Río Turbio SA',
  'parent': 'Yacimientos Carboníferos de Río Turbio SA',
  'capacity': '120',
  'status': 'construction',
  'year': '2023',
  'subnational': 'Santa Cruz',
  'region': 'Americas',
  'lat': -51.54601,
  'lng': -72.23126},
 {'id': 4,
  'country': 'Argentina',
  

In [5]:
json_data[0]

{'id': 2,
 'country': 'Argentina',
 'url': 'https://www.gem.wiki/R%C3%ADo_Turbio_power_station',
 'plant': 'Río Turbio power station',
 'unit': 'Unit 1',
 'plant_local': 'Central Térmica Río Turbio',
 'owner': 'Yacimientos Carboníferos de Río Turbio SA',
 'parent': 'Yacimientos Carboníferos de Río Turbio SA',
 'capacity': '120',
 'status': 'operating',
 'year': '2022',
 'subnational': 'Santa Cruz',
 'region': 'Americas',
 'lat': -51.54601,
 'lng': -72.23126}

In [6]:
def get_df_from_json_geo(json_file):
    lista =[]
    for i in json_file:
        country = i['country']
        url = i['url']
        plant = i['plant']
        unit = i['unit']
        plant_local = i['plant_local']
        owner = i['owner']
        parent = i['parent']
        capacity = i['capacity']
        status = i['status']
        year = i['year']
        subnational = i['subnational']
        region = i['region']
        lat = i['lat']
        lon = i['lng']
        dicc = {'project':plant,
                'type':'coal',
                'country': country,
                'unit':unit,
                'plant_local':plant_local,
                'status':status,
                'capacity':capacity,
                'subnational':subnational,
                'region':region,
                'start_year':year,
                'owner':owner,
                'parent':parent,
                'lat':lat,
                'lng':lon,
                'url':url,}
        lista.append(dicc)
    df = pd.DataFrame(lista)
    return df

In [7]:
coal_plants = get_df_from_json_geo(json_data)

In [8]:
coal_plants[coal_plants["country"]== "Spain"].shape

(51, 15)

In [9]:
coal_plants.columns = coal_plants.columns.str.strip()

In [10]:
coal_plants.to_csv("starting_data/global/coal_plants.csv", index=False)

In [11]:
coal_plants.sample()

,project,type,country,unit,plant_local,status,capacity,subnational,region,start_year,owner,parent,lat,lng,url
11292,Novaky power station,coal,Slovakia,Phase B Unit 1,,operating,110,Trenčín,Europe,1964,Slovenské Elektrárne AŞ,Ministry of Economy (Slovakia) [34.0%]; EP Slo...,48.69882,18.53346,https://www.gem.wiki/Novaky_power_station


## OIL AND GAS

In [12]:
oil_gas=pd.read_csv("https://greeninfo-network.github.io/global-gas-plant-tracker/static/data/data.csv")

In [13]:
oil_gas['type'] = 'oil_gas'

In [14]:
oil_gas.columns = oil_gas.columns.str.strip()

In [15]:
oil_gas.sample()

,project,project_loc,unit,province,country,region,status,fuel_type,capacity,technology,start_year,owner,parent,lat,lng,url,type
4108,Haritha power station,محطة كهرباء الهارثة,3,Basrah,Iraq,Asia,mothballed,"natural gas, crude oil",200.0,ST,1979,Ministry of Electricity (Iraq),Ministry of Electricity (Iraq),30.67749,47.754375,https://www.gem.wiki/Haritha_power_station,oil_gas


In [16]:
oil_gas.to_csv("starting_data/global/oil_gas.csv",index=False)

## SOLAR

In [17]:
solar =pd.read_csv("https://greeninfo-network.github.io/solar-power-tracker/static/data/data.csv")

In [18]:
solar['type'] = 'solar'

In [19]:
solar.sample()

,project,unit,project_local,capacity,units,rating,status,start_year,operator,owner,country,url,lat,lng,accuracy,type,region,subregion
11086,Ningxia Shapotou (Trina ) solar farm,NaN,天合光能沙坡头区200MWp光伏复合发电项目,200.0,NaN,MWp/dc,operating,2022.0,Zhongwei Hejun New Energy CO LTD,Changzhou Hejun New Energy CO LTD,China,https://gem.wiki/Ningxia_Shapotou_(Trina_)_sol...,37.1908,104.7138,approximate,solar,Asia,Eastern Asia


In [20]:
solar.columns = solar.columns.str.strip()
solar = solar.drop(columns='unit')

In [21]:
solar = solar.rename(columns={'units': 'unit'})

In [22]:
solar.sample()

,project,project_local,capacity,unit,rating,status,start_year,operator,owner,country,url,lat,lng,accuracy,type,region,subregion
13519,Xinjiang Kezhou Atushi Zhongxing solar farm,中兴能源源光克州阿图什40兆瓦光伏并网发电项目,40.0,NaN,unknown,operating,2016.0,Atushi Yuanguang Energy CO LTD,China Nuclear Exchange Energy CO LTD,China,https://gem.wiki/Xinjiang_Kezhou_Atushi_Zhongx...,42.4805,85.4633,approximate,solar,Asia,Eastern Asia


In [23]:
solar.to_csv("starting_data/global/solar.csv",index=False)

## WIND

In [24]:
wind =pd.read_csv("https://greeninfo-network.github.io/wind-power-tracker/static/data/data.csv")

In [25]:
wind['type'] = 'wind'

In [26]:
wind.sample()

,project,unit,project_local,capacity,units,type,status,start_year,operator,owner,country,url,lat,lng,accuracy,region,subregion
16479,Vilalba Dels Arcs wind farm,NaN,NaN,24.0,NaN,wind,operating,2010.0,Parc Eòlic Vilalba Dels Arcs SLU,Finerge SA,Spain,https://gem.wiki/Vilalba_Dels_Arcs_wind_farm,41.613,-0.2501,exact,Europe,Southern Europe


In [27]:
wind.columns = wind.columns.str.strip()
wind.drop(columns = 'unit', inplace = True)

In [28]:
wind = wind.rename(columns={'units': 'unit'})

In [29]:
wind.sample()

,project,project_local,capacity,unit,type,status,start_year,operator,owner,country,url,lat,lng,accuracy,region,subregion
1553,Serra Das Almas wind farm,Complexo Eólico Serra das Almas,28.0,MW,wind,announced,NaN,PEC Energia,PEC Energia,Brazil,https://gem.wiki/Serra_Das_Almas_wind_farm,-14.7681,-42.6574,approximate,Americas,Latin America and the Caribbean


In [30]:
wind.to_csv("starting_data/global/wind.csv",index=False)

## HYDROPOWER

In [31]:
hydro =pd.read_csv("https://greeninfo-network.github.io/global-hydropower-tracker/static/data/data.csv")

In [32]:
hydro['type'] = 'hydropower'

In [33]:
hydro.columns = hydro.columns.str.strip()
hydro = hydro.rename(columns={'countries': 'country'})

In [34]:
hydro.columns = hydro.columns.str.strip()
hydro = hydro.rename(columns={'units': 'unit'})

In [35]:
hydro.sample()

,project,capacity,unit,type,status,start_year,owner,operator,lat,lng,accuracy,url,country,region,subregion
621,Baishan hydroelectric plant,300,MW,hydropower,operating,2006.0,State Grid Xinyuan Hydropower CO LTD,Baishan Power Plant,42.7259,127.2264,exact,https://gem.wiki/Baishan_hydroelectric_plant,China,Asia,Eastern Asia


In [36]:
hydro.to_csv("starting_data/global/hydro.csv",index=False)

## GEOTHERMAL

In [37]:
geother =pd.read_csv("https://greeninfo-network.github.io/geothermal-power-tracker/static/data/data.csv")

In [38]:
geother['type'] = 'geothermal'

In [39]:
geother.columns = geother.columns.str.strip()

In [40]:
geother.sample()

,project,unit,project_local,type,capacity,units,status,start_year,operator,owner,country,url,lat,lng,accuracy,region,subregion
99,Muarah Laboh geothermal plant,2,NaN,geothermal,75,MW,construction,2023.0,PT Supreme Energy,ENGIE [18.5%]; Sumitomo CORP [18.5%]; PT Supre...,Indonesia,https://gem.wiki/Muarah_Laboh_geothermal_plant,-1.6268,101.1296,exact,Asia,South-eastern Asia


In [41]:
geother.to_csv("starting_data/global/geother.csv",index=False)

## BIOENERGY

In [42]:
bio =pd.read_csv("https://greeninfo-network.github.io/bioenergy-power-tracker/static/data/data.csv")

In [43]:
bio['type'] = 'bioenergy'

In [44]:
bio.columns = bio.columns.str.strip()

In [45]:
bio.sample()

,project,unit,fuel1,fuel2,fuel3,capacity,status,start_year,conversion,owner,operator,country,url,lat,lng,accuracy,conversion_year,type
550,Prattville Mill power station,GEN1,bioenergy_-_paper_mill_wastes,bioenergy_-_wood_and_other_biomass,fossil_gas_-_natural_gas,41.0,operating,1967.0,no conversion,International Paper CO,International Paper CO,United States,https://gem.wiki/Prattville_Mill_power_station,32.4187,-86.4718,exact,NaN,bioenergy


In [46]:
bio.to_csv("starting_data/global/bio.csv",index=False)

## NUCLEAR

In [47]:
nuclear =pd.read_csv("https://greeninfo-network.github.io/nuclear-power-tracker/static/data/data.csv")

In [48]:
nuclear['type'] = 'nuclear'

In [49]:
nuclear.columns = nuclear.columns.str.strip()

In [50]:
nuclear.sample()

,project,unit,capacity,type,status,start_year,owner,operator,country,url,lat,lng,accuracy
227,Nanyang nuclear power plant,3,1250.0,nuclear,cancelled,NaN,China Nuclear Power CO LTD [51%]; State Power ...,CNNC Henan Nuclear Power CO LTD,China,https://gem.wiki/Nanyang_nuclear_power_plant,33.3531,112.6416,Approximate


In [51]:
nuclear.to_csv("starting_data/global/nuclear.csv",index=False)

# MAKE A BIG DF

In [52]:
df_list = [coal_plants, oil_gas, solar, wind, hydro, geother, bio, nuclear]

In [53]:
common_columns = (df_list[0].columns)
for df in df_list[1:]:
    common_columns = common_columns.intersection(df.columns)

In [54]:
list_subset  = list(common_columns)

In [55]:
list_subset

['project',
 'type',
 'country',
 'unit',
 'status',
 'capacity',
 'start_year',
 'owner',
 'lat',
 'lng',
 'url']

In [56]:
def concat_df_in_common_columnsj(df_list):
    new_df_list = []
    for df in df_list:
        df_subset= df[list_subset]
        new_df_list.append(df_subset)
    big_df = pd.concat(new_df_list)

    return big_df

In [57]:
prueba=wind[list_subset]

In [59]:
big_df = concat_df_in_common_columnsj(df_list)

In [60]:
big_df.shape

(81402, 11)

In [61]:
spain_global['unit'] = 'MWh'

# SPAIN

In [62]:
spain_global = big_df[big_df['country']== 'Spain']

In [63]:
spain_global.head()

,project,type,country,unit,status,capacity,start_year,owner,lat,lng,url
11582,Aboño power station,coal,Spain,Unit 1,operating,360,1974,HC Energia,43.55356,-5.72185,https://www.gem.wiki/Abo%C3%B1o_power_station
11583,Aboño power station,coal,Spain,Unit 2,operating,556.2,1985,HC Energia,43.55356,-5.72185,https://www.gem.wiki/Abo%C3%B1o_power_station
11584,Alcúdia power station,coal,Spain,Phase 2 Unit 1,retired,125,1981,GESA-Endesa,39.81075,3.09141,https://www.gem.wiki/Alcúdia_power_station
11585,Alcúdia power station,coal,Spain,Phase 2 Unit 2,retired,125,1982,GESA-Endesa,39.81075,3.09141,https://www.gem.wiki/Alcúdia_power_station
11586,Alcúdia power station,coal,Spain,Phase 2 Unit 3,operating,130,1997,GESA-Endesa,39.81075,3.09141,https://www.gem.wiki/Alcúdia_power_station


In [64]:
spain_global.shape

(2987, 11)

In [65]:
spain_global.to_csv("starting_data/global/spain_global.csv",index=False)

# INTRODUCING CCAA IN SPAIN_GLOBAL

In [66]:
pd.set_option('display.max_columns',None)

In [67]:
spain_global = pd.read_csv("starting_data/global/spain_global.csv")

In [68]:
spain_global.columns=spain_global.columns.str.strip()

In [69]:
spain_global.shape

(2987, 11)

In [70]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [93]:
def get_autonomous_community_from_geojson(lat, lng, geojson_path):
    # Read the GeoJSON file into a GeoDataFrame
    gdf_autonomous_communities = gpd.read_file("data/spain-communities.geojson")

    # Create a GeoDataFrame with the point geometry for the given coordinates
    gdf_point = gpd.GeoDataFrame(geometry=gpd.points_from_xy([lng], [lat]))
    gdf_point.crs = {'init': 'epsg:4326'}

    # Perform a spatial join to associate the point with the corresponding autonomous community
    result = gpd.sjoin(gdf_point, gdf_autonomous_communities, how='left', op='within')

    # Extract the autonomous community name from the result
    autonomous_community = result['name'].iloc[0] if not result.empty else 'Unknown'

    return autonomous_community

In [96]:
spain_global['aacc'] = spain_global.apply(lambda row: get_autonomous_community_from_geojson(row['lat'], row['lng'], 'path/to/your/autonomous_communities.geojson'), axis=1)

C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pandas\core\apply.py:979: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  results[i] = self.func(v, *self.args, **self.kwargs)
C:\Users\psaez\AppData\Local\Temp\ipykernel_35812\407370565.py:10: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  result = gpd

C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pandas\core\apply.py:979: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  results[i] = self.func(v, *self.args, **self.kwargs)
C:\Users\psaez\AppData\Local\Temp\ipykernel_35812\407370565.py:10: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  result = gpd

C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pandas\core\apply.py:979: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  results[i] = self.func(v, *self.args, **self.kwargs)
C:\Users\psaez\AppData\Local\Temp\ipykernel_35812\407370565.py:10: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  result = gpd

C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pandas\core\apply.py:979: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  results[i] = self.func(v, *self.args, **self.kwargs)
C:\Users\psaez\AppData\Local\Temp\ipykernel_35812\407370565.py:10: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  result = gpd

C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pandas\core\apply.py:979: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  results[i] = self.func(v, *self.args, **self.kwargs)
C:\Users\psaez\AppData\Local\Temp\ipykernel_35812\407370565.py:10: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  result = gpd

...

C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pandas\core\apply.py:979: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  results[i] = self.func(v, *self.args, **self.kwargs)
C:\Users\psaez\AppData\Local\Temp\ipykernel_35812\407370565.py:10: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  result = gpd

C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pandas\core\apply.py:979: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  results[i] = self.func(v, *self.args, **self.kwargs)
C:\Users\psaez\AppData\Local\Temp\ipykernel_35812\407370565.py:10: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  result = gpd

C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pandas\core\apply.py:979: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  results[i] = self.func(v, *self.args, **self.kwargs)
C:\Users\psaez\AppData\Local\Temp\ipykernel_35812\407370565.py:10: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  result = gpd

C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\psaez\miniconda3\envs\ironhack\Lib\site-packages\pandas\core\apply.py:979: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  results[i] = self.func(v, *self.args, **self.kwargs)
C:\Users\psaez\AppData\Local\Temp\ipykernel_35812\407370565.py:10: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  result = gpd

KeyboardInterrupt: 

In [108]:
spain_global.to_csv("data/spain_global.csv",index=False)